<a href="https://colab.research.google.com/github/BernieT7/AI/blob/main/%E9%A3%9F%E8%AD%9C%E7%94%A2%E7%94%9F%E5%99%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 輸入任意食材，設定創意度，AI會幫你構思出一道符合創意度的料理料理名稱以及食譜
!pip install openai
!pip install python-dotenv

In [ ]:
from openai import OpenAI
from dotenv import dotenv_values
config = dotenv_values('.env')
client = OpenAI(api_key=config["API_KEY"])

In [ ]:
# 根據食材，創意度設計菜名
def generate_food_name(ingredient, temperature):
  prompt = f"""
    根據以下食材，產生一道菜名

    ###
    小黃瓜 蒜
    ###
    蒜味涼拌小黃瓜

    ###
    九層塔 烏龍麵 沙茶醬 雞肉
    ###
    塔香沙茶炒烏龍麵

    ###
    {ingredient}
    ###
  """

  res = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    max_tokens=100,
    temperature=temperature
  )

  return res.choices[0].text

In [ ]:
# 根據食材，產生食譜
def generate_recipe(ingredient, dish):
  prompt = f"""
    根據以下菜名以及食材，產生食譜

    輸出格式 : json array

    ###
    菜名:蒜味涼拌小黃瓜
    食材:小黃瓜
    ###
    [
      "1.小黃瓜洗淨切段拍扁，加鹽抓勻靜置30分鐘再倒掉瀝出的水份，小黃瓜洗淨切段拍扁，加鹽抓勻靜置30分鐘再倒掉瀝出的水份",
      "2.加入所有調味料放冰箱冷藏一晚，隔天就可以吃囉"
    ]

    ###
    菜名:椒鹽杏鮑菇
    食材:杏鮑菇 蒜末 九層塔
    ###
    [
      "1.準備食材、蒜切末、蔥切蔥花、辣椒剪小丁備用",
      "2.杏鮑菇切片切5到10公分，取容器放入杏鮑菇倒入蛋液混合抓勻，灑上地瓜粉（酥炸粉）粉不用灑太多適量就好",
      "3.起熱鍋下3湯匙油，油溫熱160度下杏鮑菇煎炸，待兩面金黃色表皮酥脆即可夾起",
      "4.九層塔一樣沾粉下去煎炸",
      "5.取個乾淨鍋子熱鍋，下蔥、蒜、辣椒炒出香味，下杏鮑菇翻炒幾下，即可起鍋完成料理"
    ]

    ###
    菜名:{dish}
    食材:{ingredient}
    ###
  """

  res = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    max_tokens=500,
    temperature=0.8
  )

  return res.choices[0].text

In [ ]:
# 根據食材，菜名以及食譜產生圖片
def generate_image(ingredient, dish, recipe):
  ingredients = ingredient.split(" ")
  prompt = f"一道叫做{dish}的料理，裡面的食材有"

  for i in ingredients:
    prompt += f"{i},"                               # 輸入食材

  prompt += "製作方式為"

  for i in recipe:
    prompt += i                                  # 輸入食譜

  res = client.images.generate(
      prompt=prompt,
      size="256x256"
  )

  return res.data[0].url

In [ ]:
!pip install gradio==3.48.0

In [ ]:
import json
# 整合上面三個函式，並在網頁上產生菜名，食譜，圖片
def generate_dish_recipe_image(ingredient, temperature):
  dish_name = generate_food_name(ingredient, temperature)
  recipe = generate_recipe(ingredient, dish_name)
  recipe = json.loads(recipe)
  dish_img = generate_image(ingredient, dish_name, recipe)

  html = f"""
    <h1>{dish_name}</h1>
    <img src='{dish_img}'/>
  """
  for i in recipe:
    html += f"<div>{i}</div>"


  return html

In [ ]:
import gradio as gr

demo = gr.Interface(
    fn=generate_dish_recipe_image,
    inputs=[
      gr.Text(label="輸入您的食材(中間用空白隔開) ex:牛肉 豆腐 雞蛋"),                   # 輸入食材
      gr.Slider(0, 2, value=1, label="創意度")                               # 輸入創意度
    ],
    outputs="html",                                               # 輸出為html
    title="食譜產生器",
    description="輸入食材產生一道菜及食譜",
    allow_flagging="never"
)

demo.launch(debug=True)